In [2]:
import chess
import chess.engine

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [3]:
from generate_data import *

KeyboardInterrupt: 

https://python-chess.readthedocs.io/en/latest/

https://python-chess.readthedocs.io/en/latest/core.html

In [3]:
class ChessDataset(Dataset):
    """Chess Positions Evaluation Dataset"""

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        position = np.array(self.df.iloc[idx, :-1])
        eval_ = np.array(self.df.iloc[idx, -1]/100)
        
        sample = {'position': torch.from_numpy(position), 'eval': torch.from_numpy(eval_)}

        return sample

In [4]:
dataset = ChessDataset(csv_file='data/stockfish_depth0d.csv')

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = nn.Conv2d(8, 8, 2) #input, output channels, kernel_size
        self.conv2 = nn.Conv2d(8, 8, 3)
        self.conv3 = nn.Conv2d(8, 8, 3)
        self.conv4 = nn.Conv2d(8, 8, 3)
        self.fc1 = nn.Linear(8, 64) 
        self.fc2 = nn.Linear(64, 1) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)   
        x = torch.reshape(x, (-1,)) # flattens into 1d tensor
        x = self.fc1(F.relu(x))
        x = self.fc2(torch.sigmoid(x))

        return x

In [6]:
net = Net()

# TRAINING DATA

In [7]:
optimizer = optim.Adam(net.parameters(), lr = 0.0009) #lr = 0.001
criterion = nn.MSELoss() 

In [8]:
trainloader = DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=0)

for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['position'].reshape(1, 8, 8, 8).float()
        labels = data['eval'].float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 8.789
[1,   400] loss: 8.691
[1,   600] loss: 9.017
[1,   800] loss: 10.533
[1,  1000] loss: 8.062
[1,  1200] loss: 9.221
[1,  1400] loss: 8.676
[1,  1600] loss: 8.512
[1,  1800] loss: 8.648
[1,  2000] loss: 9.177
[1,  2200] loss: 8.028
[1,  2400] loss: 8.089
[1,  2600] loss: 7.578
[1,  2800] loss: 8.287
[1,  3000] loss: 8.259
[1,  3200] loss: 8.061
[1,  3400] loss: 6.134
[1,  3600] loss: 5.985
[1,  3800] loss: 7.269
[1,  4000] loss: 7.517
[1,  4200] loss: 7.960
[1,  4400] loss: 6.161
[1,  4600] loss: 7.046
[1,  4800] loss: 6.975
[1,  5000] loss: 6.317
[1,  5200] loss: 5.977
[1,  5400] loss: 7.039
[1,  5600] loss: 5.621
[1,  5800] loss: 5.027
[1,  6000] loss: 7.281
[1,  6200] loss: 6.687
[1,  6400] loss: 6.283
[1,  6600] loss: 5.623
[1,  6800] loss: 6.367
[1,  7000] loss: 7.091
[1,  7200] loss: 4.875
[1,  7400] loss: 6.113
[1,  7600] loss: 6.430
[1,  7800] loss: 6.907
[1,  8000] loss: 6.190
[1,  8200] loss: 5.452
[1,  8400] loss: 4.741
[1,  8600] loss: 6.641
[1,  8800]

In [13]:
def compare_eval():
    board = generate_rand_board()
    torch_board = torch.from_numpy(convert_board(board).reshape(1, 8, 8, 8)).float()
    net_eval = net(torch_board)
    sf_eval = stockfish_eval(board)
    #print(board)
    print(f"NN: {net_eval} Stockfish: {sf_eval}")

In [14]:
compare_eval()

NameError: name 'generate_rand_board' is not defined